In [14]:
from pycalphad import Database
from pycalphad.variables import Species

dbf = Database('Al-Cu-Zr_Zhou.tdb')

In [35]:
import itertools
import string
import numpy as np

def genericize(mapping, const_array):
    new_array = []
    for subl in const_array:
        new_array.append(tuple(mapping[x] for x in subl))
    return tuple(new_array)
def sigfigs(x, n):
    """Round x to n significant digits"""
    if x != 0:
        return np.around(x, -(np.floor(np.log10(np.abs(x)))).astype(np.int) + (n - 1))
    else:
        return x

for combo in itertools.combinations(dbf.elements - {'VA', '/-'}, 2):
    combo_str = sorted(set(combo) | {'VA'})
    generic_combo = list(string.ascii_uppercase[:len(combo)])
    combo = [Species(x) for x in combo_str]
    combo_dict = dict(zip([c for c in combo if c.number_of_atoms > 0], string.ascii_uppercase))
    combo_dict[Species('VA')] = 'VA'
    allowed_phases = set()
    # TODO: Categorize each phase? solid solution, intermetallic, complex intermetallic, liquid, etc.
    for phase_name, phase_obj in dbf.phases.items():
        include_phase = True
        for sublattice in phase_obj.constituents:
            if len(set(sublattice).intersection(combo)) == 0:
                # None of the components in a sublattice are active
                # We cannot build a model of this phase
                include_phase = False
                break
        if include_phase:
            allowed_phases |= {phase_name}
    for param in dbf._parameters.all():
        if param['phase_name'] in allowed_phases and param['parameter_type'] in ['G', 'L']:
            active_pure_elements = [list(x.constituents.keys()) for subl in param['constituent_array'] for x in subl]
            active_pure_elements = [el.upper() for constituents in active_pure_elements for el in constituents]
            active_pure_elements = set(active_pure_elements)
            if not active_pure_elements.issubset(combo_str):
                continue
            ratio_sum = sum(dbf.phases[param['phase_name']].sublattices)
            reduced_ratios = tuple(sigfigs(c/ratio_sum, 3) for c in dbf.phases[param['phase_name']].sublattices)
            id_tuple = (generic_combo, param['phase_name'], reduced_ratios,
                        param['parameter_order'],
                        genericize(combo_dict, param['constituent_array'])
                       )
            print(id_tuple)

(['A', 'B'], 'LIQUID', (1.0,), 0, (('A',),))
(['A', 'B'], 'LIQUID', (1.0,), 0, (('B',),))
(['A', 'B'], 'LIQUID', (1.0,), 0, (('A', 'B'),))
(['A', 'B'], 'LIQUID', (1.0,), 1, (('A', 'B'),))
(['A', 'B'], 'LIQUID', (1.0,), 2, (('A', 'B'),))
(['A', 'B'], 'AL2CU', (0.66700000000000004, 0.33300000000000002), 0, (('A',), ('A',)))
(['A', 'B'], 'AL2ZR', (0.66700000000000004, 0.33300000000000002), 0, (('A',), ('B',)))
(['A', 'B'], 'AL2ZR3', (0.40000000000000002, 0.59999999999999998), 0, (('A',), ('B',)))
(['A', 'B'], 'AL3ZR2', (0.59999999999999998, 0.40000000000000002), 0, (('A',), ('B',)))
(['A', 'B'], 'AL3ZR4', (0.42899999999999999, 0.57099999999999995), 0, (('A',), ('B',)))
(['A', 'B'], 'AL3ZR5', (0.375, 0.625), 0, (('A',), ('B',)))
(['A', 'B'], 'AL4ZR5', (0.44400000000000001, 0.55600000000000005), 0, (('A',), ('B',)))
(['A', 'B'], 'ALZR', (0.5, 0.5), 0, (('A',), ('B',)))
(['A', 'B'], 'ALZR2', (0.33300000000000002, 0.66700000000000004), 0, (('A',), ('B',)))
(['A', 'B'], 'ALZR3', (0.25, 0.75), 

In [22]:
import string
string.ascii_uppercase

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'